To run via slurm, first output the ipynb notebook to python using `jupyter nbconvert --to script optimization_model.ipynb`

In [257]:
import time
import pandas as pd
import math

## Load precomputed combinations to save computation time

In [215]:
comb_df = pd.read_pickle("../precompute_combinations/precomputed_combinations_df.pkl")

def choose(n, k):
    index = n * 1201 + k
    return comb_df.at[index, "n_choose_k"]

## Probabilities based on our data analysis

In [248]:
# Probability of a goal in a given 10 second period during 5v5
p_a1 = 5.18 * 10**-3
p_b1 = p_a1

# Probability of the other team scoring in a given 10 second period during 6v5 increases
p_b2 = 8.51 * 10**-2

# Probability of our team scoring during 6v5 also increases, though not as much
p_a2 = 3.10 * 10**-2

## Inputs to model

In [299]:
# Current score
a_0 = 1
b_0 = 2

# Time remaining in game
seconds_left = 3 * 60

## Brute force every choice of $s$

Each team is allowed to score either 0 or 1 goals in each 10 second period.

Can launch via slurm on SSCC clusters to speed up

In [300]:
decisions_df = pd.DataFrame(columns = [
        'pull_time',
        'prob_success',
        'compute_time'])

results = []

T = int(seconds_left / 10)  # number of 10 second periods left in the game

#  Try out each possible time to pull the goalie
for s in range(0, T + 1):

    # Timing for performance metrics
    start_time = time.time()

    prob_success = 0

    # Tracking num_arrangements for complexity considerations
    num_arrangements = 0

    # Number of goals scored by other team before pulling the goalie
    for g_b1 in range(0, s+1):
        prob_gb1 = choose(s, g_b1) * (p_b1 ** g_b1) * ((1 - p_b1) ** (s - g_b1))

        # Number of goals scored by our team before pulling the goalie
        for g_a1 in range(0, s+1):
            prob_ga1 = choose(s, g_a1) * (p_a1 ** g_a1) * ((1 - p_a1) ** (s - g_a1))

            # Number of goals scored by other team after pulling the goalie
            for g_b2 in range(0, T - s+1):
                prob_gb2 = choose(T - s, g_b2) * (p_b2 ** g_b2) * ((1 - p_b2) ** (T - s - g_b2))

                # Number of goals scored by our team after pulling the goalie
                for g_a2 in range(0, T - s+1):
                    num_arrangements += 1
                    prob_ga2 = choose(T - s, g_a2) * (p_a2 ** g_a2) * ((1 - p_a2) ** (T - s - g_a2))

                    # Probability of this event
                    prob = prob_gb1 * prob_ga1 * prob_gb2 * prob_ga2

                    # Update the score
                    a = a_0 + g_a1 + g_a2
                    b = b_0 + g_b1 + g_b2

                    # Success if we win or tie
                    success = (a >= b)

                    prob_success = prob_success + prob * success
                    # display(f"    prob_ga1={prob_ga1}, prob_gb1={prob_gb1}, prob_ga2={prob_ga2}, prob_gb2={prob_gb2}")
                    # display(
                    #     f"  ga1={g_a1}, gb1={g_b1}, ga2={g_a2}, gb2={g_b2}, success: {success}, probability: {prob}"
                    # )

    prob_success = round(prob_success * 100, 4)

    end_time = time.time()
    elapsed_time = end_time - start_time
    elapsed_time = round(elapsed_time, 2)

    # display(f"Pull time: {s * 10} seconds, num_arrangements: {num_arrangements} Probability of success: {prob_success}, Compute time: {elapsed_time}")

    results.append({
        "pull_time": s*10,
        "prob_success": prob_success,
        "compute_time": elapsed_time
    })

decisions_df = pd.DataFrame(results)
display(decisions_df)

if int(seconds_left % 60) == 0:
    formatted_time_left = f"{int(seconds_left / 60)}"
else:
    formatted_time_left = f"{math.floor(seconds_left / 60)}:{int(seconds_left % 60):02}"
decisions_df.to_pickle(f"results/{formatted_time_left}minleft_{a_0}to{b_0}.pkl")

,pull_time,prob_success,compute_time
0,0,12.7921,0.00
1,10,12.9720,0.00
2,20,13.1403,0.01
3,30,13.2941,0.01
4,40,13.4297,0.01
5,50,13.5426,0.02
6,60,13.6274,0.02
7,70,13.6780,0.02
8,80,13.6866,0.02
9,90,13.6440,0.02


## Select the optimal choice

In [301]:
optimal_index = decisions_df["prob_success"].idxmax()
optimal_time = decisions_df.loc[optimal_index, "pull_time"]
optimal_prob = decisions_df.loc[optimal_index, "prob_success"]

print(f"Optimal time to pull the goalie: {optimal_time} seconds, resulting in a probability of {optimal_prob} of winning or tying the game.")

Optimal time to pull the goalie: 80 seconds, resulting in a probability of 13.6866 of winning or tying the game.
